<a href="https://colab.research.google.com/github/shawkoike/colab/blob/main/%E6%96%87%E7%AB%A0%E8%A6%81%E7%B4%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
path = '/content/drive/MyDrive/google_colaboratory/document_summarization/'
if not os.path.exists(path):
    os.makedirs(path)

%cd /content/drive/MyDrive/google_colaboratory/document_summarization/

Mounted at /content/drive
/content/drive/MyDrive/google_colaboratory/document_summarization


In [ ]:
pip install neologdn

     |████████████████████████████████| 57 kB 2.6 MB/s 
  Created wheel for neologdn: filename=neologdn-0.5.1-cp37-cp37m-linux_x86_64.whl size=172965 sha256=4bbfe1466641631fe238c90cd26cccebfe65c159051ad95aadcbad5bd51a905e
  Stored in directory: /root/.cache/pip/wheels/19/15/5c/55b33d02e16129ef81313e4c86e473d6dd1cecf7317a525a9b
Successfully built neologdn


In [ ]:
pip install sentencepiece==0.1.91

     |████████████████████████████████| 1.1 MB 5.3 MB/s 


In [ ]:
pip install transformers

     |████████████████████████████████| 3.4 MB 5.3 MB/s 
     |████████████████████████████████| 67 kB 5.4 MB/s 
     |████████████████████████████████| 895 kB 60.2 MB/s 
     |████████████████████████████████| 596 kB 57.4 MB/s 
     |████████████████████████████████| 3.3 MB 53.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from pathlib import Path
import re
import math
import time
import copy
from tqdm import tqdm
import pandas as pd
import tarfile
import neologdn
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch import optim
from torch.utils.data import DataLoader
from transformers import T5ForConditionalGeneration, T5Tokenizer

In [ ]:
MODEL_NAME = "sonoisa/t5-base-japanese"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

max_length_src = 200
max_length_target = 100

batch_size_train = 8
batch_size_valid = 8

epochs = 10
patience = 20

In [ ]:
body_data = pd.read_csv('./body_data.csv')
summary_data = pd.read_csv('./summary_data.csv')

pd.merge(
    body_data.query('text.notnull()', engine='python').rename(columns={'text': 'body'}),
    summary_data.rename(columns={'text': 'summary'}),
    on='article_id', how='inner'
).sort_values('article_id').head(10)

,article_id,title,body,summary
0,11104959,【蕎麦】「丸の内・東京駅」エリアで発見! うまくて安い“蕎麦ランチ”の名店7,\n var amp_med = '2000689';\n ...,東京駅周辺の安くて美味しい「蕎麦ランチ」の名店を紹介している
1,11104959,【蕎麦】「丸の内・東京駅」エリアで発見! うまくて安い“蕎麦ランチ”の名店7,\n var amp_med = '2000689';\n ...,「越後そば 東京店」では、ミニかき揚げ丼セットがおすすめと筆者
2,11104959,【蕎麦】「丸の内・東京駅」エリアで発見! うまくて安い“蕎麦ランチ”の名店7,\n var amp_med = '2000689';\n ...,その他には、「手打ちそば 石月」「酢重正之 楽」「鎌倉 一茶庵 丸山」など
7,11863484,長友所属インテル、マンチーニ監督の退任が決定的に…後任はデ・ブール氏か,\n 日本代表DF長友佑都が所属するインテルは、ロベルト・マンチーニ監督との契約を解消し、後...,前アヤックス指揮官のフランク・デ・ブール氏を新監督として招へいする模様
6,11863484,長友所属インテル、マンチーニ監督の退任が決定的に…後任はデ・ブール氏か,\n 日本代表DF長友佑都が所属するインテルは、ロベルト・マンチーニ監督との契約を解消し、後...,インテルのマンチーニ監督の退任が決定的になったようだと報じられた
8,11863484,長友所属インテル、マンチーニ監督の退任が決定的に…後任はデ・ブール氏か,\n 日本代表DF長友佑都が所属するインテルは、ロベルト・マンチーニ監督との契約を解消し、後...,マンチーニ監督の契約解除金は、約2億8000万円になると伝えられている
4,11870207,ここがヘンだよ、五輪サッカー！​5つの「謎ルール」,\nオリンピックのサッカー競技には様々な制約が課せられている。 そこで今回は、編集部Sが考え...,23歳以下という年齢制限は、W杯を開催するFIFAとの利害の対立で生まれた
3,11870207,ここがヘンだよ、五輪サッカー！​5つの「謎ルール」,\nオリンピックのサッカー競技には様々な制約が課せられている。 そこで今回は、編集部Sが考え...,オリンピックのサッカー競技の謎ルールを解説している
5,11870207,ここがヘンだよ、五輪サッカー！​5つの「謎ルール」,\nオリンピックのサッカー競技には様々な制約が課せられている。 そこで今回は、編集部Sが考え...,FIFAはプロ選手の五輪出場に難色を示し、妥協案として23歳以下限定となった
12,11883769,メッシ、アルゼンチン代表引退を撤回,\nリオネル・メッシがアルゼンチン代表引退を撤回した。『ESPN』などが報じている。9月のロ...,メッシがアルゼンチン代表引退を撤回したと「ESPN」などが報じた


In [ ]:
def join_text(x, add_char='。'):
    return add_char.join(x)

def preprocess_text(text):
    text = re.sub(r'[\r\t\n\u3000]', '', text)
    text = neologdn.normalize(text)
    text = text.lower()
    text = text.strip()
    return text

summary_data = summary_data.query('text.notnull()', engine='python').groupby(
    'article_id'
).agg({'text': join_text})

body_data = body_data.query('text.notnull()', engine='python')

data = pd.merge(
    body_data.rename(columns={'text': 'body_text'}),
    summary_data.rename(columns={'text': 'summary_text'}),
    on='article_id', how='inner'
).assign(
    body_text=lambda x: x.body_text.map(lambda y: preprocess_text(y)),
    summary_text=lambda x: x.summary_text.map(lambda y: preprocess_text(y))
)

In [ ]:
def convert_batch_data(train_data, valid_data, tokenizer):

    def generate_batch(data):

        batch_src, batch_tgt = [], []
        for src, tgt in data:
            batch_src.append(src)
            batch_tgt.append(tgt)

        batch_src = tokenizer(
            batch_src, max_length=max_length_src, truncation=True, padding="max_length", return_tensors="pt"
        )
        batch_tgt = tokenizer(
            batch_tgt, max_length=max_length_target, truncation=True, padding="max_length", return_tensors="pt"
        )

        return batch_src, batch_tgt

    train_iter = DataLoader(train_data, batch_size=batch_size_train, shuffle=True, collate_fn=generate_batch)
    valid_iter = DataLoader(valid_data, batch_size=batch_size_valid, shuffle=True, collate_fn=generate_batch)

    return train_iter, valid_iter

tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME, is_fast=True)

X_train, X_test, y_train, y_test = train_test_split(
    data['body_text'], data['summary_text'], test_size=0.2, random_state=42, shuffle=True
)

train_data = [(src, tgt) for src, tgt in zip(X_train, y_train)]
valid_data = [(src, tgt) for src, tgt in zip(X_test, y_test)]

train_iter, valid_iter = convert_batch_data(train_data, valid_data, tokenizer)

Downloading:   0%|          | 0.00/785k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.74k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.91k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/710 [00:00<?, ?B/s]

In [ ]:
class T5FineTuner(nn.Module):
    
    def __init__(self):
        super().__init__()

        self.model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)

    def forward(
        self, input_ids, attention_mask=None, decoder_input_ids=None,
        decoder_attention_mask=None, labels=None
    ):
        return self.model(
            input_ids,
            attention_mask=attention_mask,
            decoder_input_ids=decoder_input_ids,
            decoder_attention_mask=decoder_attention_mask,
            labels=labels
        )

In [ ]:
def train(model, data, optimizer, PAD_IDX):
    
    model.train()
    
    loop = 1
    losses = 0
    pbar = tqdm(data)
    for src, tgt in pbar:
                
        optimizer.zero_grad()
        
        labels = tgt['input_ids'].to(device)
        labels[labels[:, :] == PAD_IDX] = -100

        outputs = model(
            input_ids=src['input_ids'].to(device),
            attention_mask=src['attention_mask'].to(device),
            decoder_attention_mask=tgt['attention_mask'].to(device),
            labels=labels
        )
        loss = outputs['loss']

        loss.backward()
        optimizer.step()
        losses += loss.item()
        
        pbar.set_postfix(loss=losses / loop)
        loop += 1
        
    return losses / len(data)

def evaluate(model, data, PAD_IDX):
    
    model.eval()
    losses = 0
    with torch.no_grad():
        for src, tgt in data:

            labels = tgt['input_ids'].to(device)
            labels[labels[:, :] == PAD_IDX] = -100

            outputs = model(
                input_ids=src['input_ids'].to(device),
                attention_mask=src['attention_mask'].to(device),
                decoder_attention_mask=tgt['attention_mask'].to(device),
                labels=labels
            )
            loss = outputs['loss']
            losses += loss.item()
        
    return losses / len(data)

In [ ]:
# model = T5FineTuner()
# model = model.to(device)

# optimizer = optim.Adam(model.parameters())

# PAD_IDX = tokenizer.pad_token_id
# best_loss = float('Inf')
# best_model = None
# counter = 1

# for loop in range(1, epochs + 1):

#     start_time = time.time()

#     loss_train = train(model=model, data=train_iter, optimizer=optimizer, PAD_IDX=PAD_IDX)

#     elapsed_time = time.time() - start_time

#     loss_valid = evaluate(model=model, data=valid_iter, PAD_IDX=PAD_IDX)

#     print('[{}/{}] train loss: {:.4f}, valid loss: {:.4f} [{}{:.0f}s] counter: {} {}'.format(
#         loop, epochs, loss_train, loss_valid,
#         str(int(math.floor(elapsed_time / 60))) + 'm' if math.floor(elapsed_time / 60) > 0 else '',
#         elapsed_time % 60,
#         counter,
#         '**' if best_loss > loss_valid else ''
#     ))

#     if best_loss > loss_valid:
#         best_loss = loss_valid
#         best_model = copy.deepcopy(model)
#         counter = 1
#     else:
#         if counter > patience:
#             break

#         counter += 1

In [ ]:
model_dir_path = Path('model')
# if not model_dir_path.exists():
#     model_dir_path.mkdir(parents=True)

# tokenizer.save_pretrained(model_dir_path)
# best_model.model.save_pretrained(model_dir_path)

In [ ]:
def generate_text_from_model(text, trained_model, tokenizer, num_return_sequences=1):

    trained_model.eval()
    
    text = preprocess_text(text)
    batch = tokenizer(
        [text], max_length=max_length_src, truncation=True, padding="longest", return_tensors="pt"
    )

    # 生成処理を行う
    outputs = trained_model.generate(
        input_ids=batch['input_ids'].to(device),
        attention_mask=batch['attention_mask'].to(device),
        max_length=max_length_target,
        repetition_penalty=8.0,   # 同じ文の繰り返し（モード崩壊）へのペナルティ
        temperature=1.0,  # 生成にランダム性を入れる温度パラメータ
        # num_beams=10,  # ビームサーチの探索幅
        # diversity_penalty=1.0,  # 生成結果の多様性を生み出すためのペナルティパラメータ
        # num_beam_groups=10,  # ビームサーチのグループ
        num_return_sequences=num_return_sequences,  # 生成する文の数
    )

    generated_texts = [
        tokenizer.decode(ids, skip_special_tokens=True, clean_up_tokenization_spaces=False) for ids in outputs
    ]

    return generated_texts

tokenizer = T5Tokenizer.from_pretrained(model_dir_path)
trained_model = T5ForConditionalGeneration.from_pretrained(model_dir_path)
trained_model = trained_model.to(device)

In [ ]:
index = 1

body = "クラウドソーシングサイトの大きな特徴といえば、「どんな人でも、仕事を請け負うためのスタートラインに立てること」ではないでしょうか。クラウドソーシングサイトでは規定の年齢を満たしていれば、基本的には誰でも会員として活動できるようになっています。しかし、実際に仕事を受注するには、生身の人間であるクライアントから信頼を勝ち得なければなりません。クラウドソーシングサイトでクライアントから信頼を得る主な方法は３つ。ひとつは、プロフィールページで自分のスキルや人柄をアピールすること。ふたつめは、案件への応募時にクライアントから信頼される対応をすること。みっつめは、受注した仕事を完遂すること。このページではひとつめの方法である「プロフィールページでのアピール」に焦点をあててみましょう。クライアントから好感をもってもらえるプロフィールの作り方について、３つの項目にまとめてみました。"

generated_texts = generate_text_from_model(
    text=body, trained_model=trained_model, tokenizer=tokenizer, num_return_sequences=1
)

print('□ 本文')
print(body)

print('□ 生成文')
print(generated_texts[0].split('。')[0])

print("--------------------")
print('\n'.join(generated_texts[0].split('。')))


□ 本文
クラウドソーシングサイトの大きな特徴といえば、「どんな人でも、仕事を請け負うためのスタートラインに立てること」ではないでしょうか。クラウドソーシングサイトでは規定の年齢を満たしていれば、基本的には誰でも会員として活動できるようになっています。しかし、実際に仕事を受注するには、生身の人間であるクライアントから信頼を勝ち得なければなりません。クラウドソーシングサイトでクライアントから信頼を得る主な方法は３つ。ひとつは、プロフィールページで自分のスキルや人柄をアピールすること。ふたつめは、案件への応募時にクライアントから信頼される対応をすること。みっつめは、受注した仕事を完遂すること。このページではひとつめの方法である「プロフィールページでのアピール」に焦点をあててみましょう。クライアントから好感をもってもらえるプロフィールの作り方について、３つの項目にまとめてみました。
□ 生成文
プロフィールページで自分のスキルや人柄をアピールすること
--------------------
プロフィールページで自分のスキルや人柄をアピールすること
そして、案件への応募時にクライアントから信頼される対応をすること
この3つに焦点をあててみましょう
クラウドソーシングサイトの大きな特徴といえば、「どんな人でも仕事が受注できる」ということではないでしょうか
また、仕事を受注するにはどのような方法があるのでしょうか
まずは「プロフィールページでのアピール」に注目してみましょう
と2つのステップ会員として活動できるようになっています

